In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm
import datetime as dt
import math as mt
from pymongo import MongoClient as mc
import IPython
import IPython.display
from sklearn.decomposition import TruncatedSVD 
from scipy.spatial import distance
import scipy.stats
# from modules.models import *
import modules.new_utils as nu
from tqdm import trange

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sn
sn.set()
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
matplotlib.rc('font', family='AppleGothic')  # MacOS
# matplotlib.rc('font', family='Malgun Gothic')  # Windows
plt.rcParams['axes.unicode_minus'] = False

In [4]:
# 1. read datas.xlsx
xlsx = nu.load_excel("datas/datas.xlsx")

In [5]:
mongo_uri = "mongodb://localhost:27017"
client = mc(mongo_uri)
db = client.crs_test
col = db.result

In [6]:
test_result = col.insert_one({
    "name": "it's test"
})

In [10]:
test_id = test_result.inserted_id

In [14]:
update_test_result = col.update_one({
    "_id": test_id,
}, {
    "$set":{
        "name": "it's test2",
        "name2": "it's 2"        
    }
})

In [17]:
col.find_one({
    "_id": test_id
})

{'_id': ObjectId('61fce9f0438795abd7d04efe'),
 'name': "it's test2",
 'name2': "it's 2"}

In [18]:
# 2. data pre-processing
p, m = nu.data_preprocessing(xlsx)

/Users/formegusto/Desktop/idoformegusto/development/research/contract-recommendation-system/prototype/notebook/modules/new_utils/data_preprocessing.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  datas_df[household_name] = household_df[col][3:].to_list()
/Users/formegusto/Desktop/idoformegusto/development/research/contract-recommendation-system/prototype/notebook/modules/new_utils/data_preprocessing.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  month_usage_df[name] = [


In [29]:
peaks = np.array([])

for idx in p.index:
    month = p.loc[idx]['month']
    peak = p.loc[idx]['peak (kW)']
    in_dict = dict({
        "month": month,
        "peak": peak
    })
    
    peaks = np.append(peaks, in_dict)
    
peaks

array([{'month': '1', 'peak': 224.0}, {'month': '2', 'peak': 224.0},
       {'month': '3', 'peak': 212.0}, {'month': '4', 'peak': 208.0},
       {'month': '5', 'peak': 152.0}, {'month': '6', 'peak': 200.0},
       {'month': '7', 'peak': 356.0}, {'month': '8', 'peak': 356.0},
       {'month': '9', 'peak': 204.0}, {'month': '10', 'peak': 192.0},
       {'month': '11', 'peak': 212.0}, {'month': '12', 'peak': 220.0}],
      dtype=object)

In [37]:
month_idx_m = m.set_index("month")
month_usages = np.array([])

for idx in month_idx_m.index:
    month = idx
    households_name = month_idx_m.columns.values
    households_kwh = month_idx_m.loc[idx].values
    in_dict = dict({
        "month": month,
        "names": households_name,
        "kwhs": households_kwh
    })
    
    month_usages = np.append(month_usages, in_dict)
    
month_usages

array([{'month': 1, 'names': array(['아파트1-101-1002', '아파트1-101-104', '아파트1-101-1102', '아파트1-101-1202',
              '아파트1-101-1302', '아파트1-101-1401', '아파트1-101-1403', '아파트1-101-1502',
              '아파트1-101-1504', '아파트1-101-1602', '아파트1-101-1604', '아파트1-101-201',
              '아파트1-101-301', '아파트1-101-303', '아파트1-101-402', '아파트1-101-404',
              '아파트1-101-502', '아파트1-101-503', '아파트1-101-504', '아파트1-101-604',
              '아파트1-101-704', '아파트1-101-803', '아파트1-101-804', '아파트1-101-901',
              '아파트1-101-904', '아파트1-102-1002', '아파트1-102-1004', '아파트1-102-104',
              '아파트1-102-1201', '아파트1-102-1302', '아파트1-102-1402', '아파트1-102-1501',
              '아파트1-102-1602', '아파트1-102-203', '아파트1-102-301', '아파트1-102-303',
              '아파트1-102-401', '아파트1-102-403', '아파트1-102-404', '아파트1-102-501',
              '아파트1-102-603', '아파트1-102-703', '아파트1-102-704', '아파트1-102-801',
              '아파트1-102-802', '아파트1-102-901', '아파트1-102-903', '아파트1-103-1002',
              '아파트1-103-

In [39]:
col.insert_one({
    "peak": list(peaks),
    "month_usage": list(month_usages)
})

InvalidDocument: cannot encode object: array(['아파트1-101-1002', '아파트1-101-104', '아파트1-101-1102', '아파트1-101-1202',
       '아파트1-101-1302', '아파트1-101-1401', '아파트1-101-1403', '아파트1-101-1502',
       '아파트1-101-1504', '아파트1-101-1602', '아파트1-101-1604', '아파트1-101-201',
       '아파트1-101-301', '아파트1-101-303', '아파트1-101-402', '아파트1-101-404',
       '아파트1-101-502', '아파트1-101-503', '아파트1-101-504', '아파트1-101-604',
       '아파트1-101-704', '아파트1-101-803', '아파트1-101-804', '아파트1-101-901',
       '아파트1-101-904', '아파트1-102-1002', '아파트1-102-1004', '아파트1-102-104',
       '아파트1-102-1201', '아파트1-102-1302', '아파트1-102-1402', '아파트1-102-1501',
       '아파트1-102-1602', '아파트1-102-203', '아파트1-102-301', '아파트1-102-303',
       '아파트1-102-401', '아파트1-102-403', '아파트1-102-404', '아파트1-102-501',
       '아파트1-102-603', '아파트1-102-703', '아파트1-102-704', '아파트1-102-801',
       '아파트1-102-802', '아파트1-102-901', '아파트1-102-903', '아파트1-103-1002',
       '아파트1-103-1004', '아파트1-103-1006', '아파트1-103-106', '아파트1-103-1102',
       '아파트1-103-